# Ejemplo 2: Modelos de regresión


### Datos que se utilizan en el ejemplo

Se utilizan los datos del total de casos positivos de Covid-19 e indicadores del mercado laboral provenientes de la **Gran Encuesta Integrada de Hogares (GEIH)** del [DANE](https://www.dane.gov.co/index.php/estadisticas-por-tema/mercado-laboral/empleo-y-desempleo/geih-historicos) de frecuencia mensual. 
    
##### Diccionario de las variables

* **year:** Año.
* **mes:** Mes.
* **divipola_depto:** Código divipola del departamento.
* **depto:** Nombre del departamento.
* **casos_covid:** Número de casos positivos de Covid-19.
* **pop_covid:** Proporción de la población del departamento que se con casos positivos de Covid-19.
* **pea:** Población economicamente activa.
* **pei:** Población economicamente inactiva.
* **tdes:** Tasa de desempleo.
* **tocu:** Tasa de ocupacion.
* **Confinamiento:** Toma el valor de 1 en los meses que hubo confinamiento nacional, 0 en otro caso.
    


## 1. Regresión lineal

La regresión lineal es un modelo estadístico de una media condicional; es decir, una forma de predecir el valor aproximado que debería tener una variable de interés $(Y)$ teniendo en cuenta el valor de las otras varianbles $(X_i)$. Se llama regresión lineal simple cuando solo hay una variable independiente y regresión lineal múltiple cuando hay más de una.

El [modelo de regresión lineal](https://www.cienciadedatos.net/documentos/py10-regresion-lineal-python) considera que, dado un conjunto de observaciones $\{ y_{i}, x_{i1}, x_{i2}, \dots , x_{nk} \}^{n}_{i=1}$, la media de la variable dependiente ,$y_{i}$, se relaciona de forma lineal con las variables dependiente o regresoras $x_{1}, \dots , x_{p}$.

$$y_{i} = \beta_{0} +  \beta_{1}x_{1} + \beta_{2}x_{2} + \dots + \beta_{k}x_{k} + \epsilon_{i}$$

La interpretaicíon de los coeficientes $(\beta_{0}, \dots, \beta_{k})$ del modelo es la misma: 
* $\beta_{0}:$ es el promedio de la variable dependiente $(y)$ manteniendo las variables dependientes $(x_{1}, \dots, x_{k})$ constantes. Este coeficiente se conoce como intercepto.
* $\beta_{k}:$ son los efectos promedios que tiene la variable dependiente $(y)$ cuando incrementa una unidad de la variable dependiente $(x_{k})$. Estos coeficientes de conocen como efecto marginal o coeficientes parciales.
* $\epsilon_{i}:$ se conoce como residuo o término error, y es la diferencia entre el valor observado de variable dependiente $(y_{i})$ y su valor predicho $(\hat{y}_{i})$ por el modelo. El error representa la parte de la variación que no puede ser explicada por las variables independientes incluidas en el modelo.

Como los valores poblacionales de los coeficientes de la regresión $(\beta_{0}, \dots, \beta_{k})$ son desconocidos, se estiman a partir de una muestra de datos $(\hat{\beta}_{0}, \dots, \hat{\beta}_{k})$. La magnitud de los coeficientes parciales de regresión no está relacionada con la importancia de cada variable dependiente, ya que esta magnitud puede variar según las unidades de medida de las variables correspondientes.

El método comúnmente empleado para realizar las estimaciones de los coeficientes es el de ***Mínimos Cuadrados Ordinarios (MCO)***, cuyo objetivo es encontrar los valores de los coeficientes de regresión que minimizan la suma de los cuadrados de las diferencias entre los valores observados y los valores predichos por el modelo.

<div>
<img src="Imagen/Regresion_Lineal.png" width="450"/> |
</div>


### Coeficiente de determinación $R^{2}$

El coeficiente de determinación $(R^2)$ indica la proporción de la variabilidad total de la variable dependiente que puede ser explicada por el modelo. Es una de las métricas más usadas para medir la calidad del ajuste del modelo, y un $R^2$ más cercano a 1 indica un mejor ajuste del modelo.

$$R^{2} = 1- \frac{Suma \space{} de \space{} Cuadrados \space{} Residuales}{Suma \space{} de \space{} Cuadrados \space{} Totales}$$

$$R^{2} = 1- \frac{\sum_{i=1}^{n}(\hat{y}_{i}-y_{i})^{2}}{\sum_{i=1}^{n}(y_{i}-\overline{y}_{i})^{2}}$$

En los modelos de regresión lineal simple, el coeficiente de determinación $(R^2)$ se corresponde con el cuadrado del coeficiente de correlación de Pearson $(\rho)$ de la variable independiente $(x_{i})$ y la variable dependiente $(y_{i})$. Sin embargo, en los modelos de regresión múltiple, esta relación entre $R^2$ y $\rho$ no se mantiene.

<div>
    <img src="Imagen/Regresion_Lineal_R2_MOD1.png" width="350"/> 
    <img src="Imagen/Regresion_Lineal_R2_MOD2.png" width="350"/>
    <img src="Imagen/Regresion_Lineal_R2_MOD3.png" width="350"/>
</div>


<div>
    <img src="Imagen/Regresion_Lineal_R2_MOD4.png" width="370"/>
</div>

### 1.1 Regresión lineal simple

Se estimará el siguiente modelo con los datos del total de casos positivos de Covid-19 e indicadores del mercado laboral:
$$TD_{it}= \beta_{0} + \beta_{1} CV_{it} + \varepsilon_{it}$$ 

donde $TD_{it}$ representa la tasa de desempleo en el departamento $i$ y el mes $t$;$CV_{it}$ don los numeros de casos positivos de Covid-19, igualmente en el departamento $i$ y el mes $t$; $\varepsilon_{it}$, es el término error.

In [18]:
#Importamos las librerias que se utilizaran

# 1. Libreria de manejos de DataFrame
import numpy as np
import pandas as pd

#2. Libreria de graficas
import matplotlib.pyplot as plt
import seaborn as sns

#3. Libreria de analisis estadistico
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy.stats import pearsonr


In [19]:
#Importamos los datos en un Dataframe y le asignamos el nombre de covid_ml
covid_ml = pd.read_csv('Data/COVID/COVID_ML_2020.csv', encoding='UTF-8')
covid_ml.head()

In [20]:
#Vemos si hay valores faltantes en la las variables del Dataframe y el tipo de datos
covid_ml.info()

In [21]:
#Generamos estadísticas descriptivas básicas
covid_ml.describe()

#### Calculamos el coeficiente de correlación

In [22]:
#Calculamos el coeficiente de correlacion entre la tasa de desempleo y el número de casos positivos de Covid-19.
coef_corr, _ = pearsonr(covid_ml['casos_covid'], covid_ml['tdes'])
print('El coeficiente de correlación entre los casos de Covid-19 y la tasa de desempleo es', coef_corr)

In [23]:
#Graficamos un grafico de dispersión para ver la relación entre las varables
plt.scatter(covid_ml['casos_covid'], covid_ml['tdes'], marker='o', color='red')
plt.text(60000, 39, f'coeficiente de correlación = {coef_corr:.4f}', fontsize=9)
plt.title('Gráfico de dispersión')
plt.ylabel('Tasa de desempleo (%)')
plt.xlabel('Casos de Covid-19')
plt.grid(False)
plt.show()

In [24]:
#Realizamos la reagresión lineal simple
modelo = smf.ols(formula = 'tdes ~ casos_covid', data = covid_ml) #Se realiza la regresion lineal
result = modelo.fit()

#Se graban los coeficientes
intercept_mod = list(result.params)[0] #El intercepto
beta_mod = list(result.params)[1] #La pendiente

#Se graba el R2 y los valores ajustados de la regresion
R2_mod = result.rsquared #El R2
TD_hat = result.fittedvalues #La prediccion del modelo

#Se imprimen los resultados
print(result.summary())

In [25]:
print('Modelo: Tasa de desempleo vs Casos positivos de Covid-19')
print(f'TD = {intercept_mod:5.3f}{beta_mod:.7f} VC')
print(f'R2 = {R2_mod:.3f}')

**¿Tiene el modelo estimado buen ajuste?**

## 


In [26]:
#Se le calcula la raíz cuadrada al R2
coef_corr_mod = np.sqrt(R2_mod)
print(coef_corr_mod)

Vemos que el la raíz cuadrada del $R^2$ es igual al valor absoluto del coeficiente de correlación entre las dos variables.

El ajuste tambien se puede ver en viendo la relación que hay entre los valores predichos por el modelo $(\hat{TD})$ y los observados $(TD)$ de la variable dependiente. Si el modelo tiene ben ajuste, los datos de concentrarían al rededor de la linea de ajuste.

In [27]:
#Se calcula una linea de ajuste
fit = np.polyfit(TD_hat,covid_ml['tdes'], 1)
fit_line = fit[1] + fit[0] * TD_hat

#Se calcula el R cuadrado para ver el ajuste de los datos predichos ocn los observados

#Se generla el grafico
plt.scatter(TD_hat, covid_ml['tdes'], marker = "o", color = 'k', label="Observado") 
plt.plot(TD_hat, fit_line, 'r', label="Ajuste") 
plt.text(s=f'$R^2$ = {R2_mod:.3f}', x=15.75, y=35, fontsize=10) 
plt.title('Modelo de regresión simple')
plt.ylabel('TD observada'), plt.xlabel('TD predicha')
plt.legend()
plt.show()

### 1.2 Regresión lineal multiple

Se estimará el siguiente modelo con los datos del total de casos positivos de Covid-19 e indicadores del mercado laboral:
$$TD_{it}= \beta_{0} + \beta_{1} CV_{it} + \beta_{2}C_{i} \varepsilon_{it}$$ 

donde $TD_{it}$ representa la tasa de desempleo en el departamento $i$ y el mes $t$;$CV_{it}$ don los numeros de casos positivos de Covid-19, igualmente en el departamento $i$ y el mes $t$;pero ahora está una variable dummy $(C_{i})$ que toma el valos de $1$ en los meses que hubo confinamiento y el valor de $0$ en otro caso; $\varepsilon_{it}$, es el término error.

In [28]:
#Realizamos la reagresión lineal multiple
modelo_mult = smf.ols(formula = 'tdes ~ casos_covid + confinamiento', data = covid_ml) #Se realiza la regresion lineal
result_mult = modelo_mult.fit()

#Se graban los coeficientes
beta_0 = list(result_mult.params)[0]
beta_1 = list(result_mult.params)[1]
beta_2 = list(result_mult.params)[2]

#Se graba el R2 y los valores ajustados de la regresion
R2_mult = result_mult.rsquared #El R2
TD_hat_mult = result_mult.fittedvalues #La prediccion del modelo

#Se imprimen los resultados
print(result_mult.summary())

El modelo con la variable dummy de confinamiento tienen un $R^2$ más alto $(0.086)$ que el modelo de la regresión simple. Estoy indica que, aunque muy bajo, el segundo modelo tienen mejor ajuste que el primero.

In [29]:
print('Modelo 2: Tasa de desempleo vs Casos positivos de Covid-19')
print(f'TD = {beta_0:5.3f}{beta_1:.7f} VC + {beta_2:5.4f}C' )
print(f'R2 = {R2_mult:.3f}')

* **¿Cúal es la interpretación de cofeciciete de la variable CV y de la variable V?**
* **¿Cómo se interpreta el $R^2$?**

### 1.3 Otro ejemplo

Para dar una idea que la regresión busca mirar comparaciones condicional en lo que pasa con variable dependiente, se realizaran dos estimaciones, una donde la variable $D=1$, otra donde la variable $D=0$ y, por último, con toda la muestra. Los datos datos utilizados fueron simulados y generados a partir de una distribución normal. Siendo $Y$ la varieble independiente, $X$ y  $Z$ las variables dependiente y $D$ la variable Dummy.

$$ Y_{i} = \beta_{0} + \beta_{1}X_{i} + \varepsilon_{i}$$ 

In [30]:
#Importamos los datos en un Dataframe y le asignamos el nombre de covid_ml
sim_data = pd.read_csv('Data/SIM_DATA.csv', encoding='UTF-8', sep = ';', decimal=',')
sim_data.head()

In [31]:
#Regresión 1: (D =1)
sim_data1 = sim_data[sim_data['D']==1]

#Realizamos la reagresión lineal multiple
modelo_1 = smf.ols(formula = 'Y ~ X', data = sim_data1) #Se realiza la regresion lineal
result_1 = modelo_1.fit()

#Se graban los coeficientes
beta1_0 = list(result_1.params)[0]
beta1_1 = list(result_1.params)[1]

#Se imprimen los resultados
print(result_1.summary())

In [32]:
#Regresión 2: (D =0)
sim_data2 = sim_data[sim_data['D']==0]

#Realizamos la reagresión lineal multiple
modelo_2 = smf.ols(formula = 'Y ~ X', data = sim_data2) #Se realiza la regresion lineal
result_2 = modelo_2.fit()

#Se graban los coeficientes
beta2_0 = list(result_2.params)[0]
beta2_1 = list(result_2.params)[1]

#Se imprimen los resultados
print(result_2.summary())

In [36]:
#Regresión 2: (toda la muestra)

#Realizamos la reagresión lineal multiple
modelo_3 = smf.ols(formula = 'Y ~ X', data = sim_data) #Se realiza la regresion lineal
result_3 = modelo_3.fit()

#Se graban los coeficientes
beta3_0 = list(result_3.params)[0]
beta3_1 = list(result_3.params)[1]

#Se imprimen los resultados
print(result_3.summary())

#### ¿Qué vemos de particular en estas tres regresiones?

El coeficiente parcial de la regresión 3, en la que no se restringe la muestra, está entre el valor del coeficiente aprcial de la regresión 1 y el de la resgresión 2. 

In [35]:
print(f'Coeficiente de la regresión 1 (D=1): {beta1_1:1.4f}')
print(f'Coeficiente de la regresión 2 (D=0): {beta2_1:1.4f}')
print(f'Coeficiente de la regresión 3:       {beta3_1:1.4f}')